In [1]:
import numpy as np
import time
from sklearn import linear_model


def soft_thresh(x, lambda_val):
    return np.sign(x) * np.maximum(np.zeros(x.size), np.abs(x) - lambda_val)


def admm_lasso(X, y, tau, max_iterations=1000, tol=1e-4):
    XX = X.T.dot(X)
    Xy = X.T.dot(y)

    p = X.shape[1]
    lambdas = np.zeros(p)
    max_rho = 5
    rho = 4

    z0, z, beta0, beta = [np.zeros(p)] * 4
    sinv = np.linalg.inv(XX + rho * np.eye(p))

    for i in range(max_iterations):
        # update beta
        beta = sinv.dot(Xy + rho * z - lambdas)

        # update z
        z = soft_thresh(beta + lambdas / rho, tau / rho)

        # update lambda
        lambdas = lambdas + rho * (beta - z)

        change = max(np.linalg.norm(beta - beta0), np.linalg.norm(z - z0))
        if change < tol or i + 1 > max_iterations:
            break
        beta0 = beta
        z0 = z
    return z

In [ ]:
# Model
n = 50
p = 400
X = np.random.normal(size=(n, p))
b = np.zeros(400)
b[300:305] = np.arange(10, 0, -2)

y = np.dot(X, b) + np.random.normal(size=(n,))
y_new = np.dot(X, b) + np.random.normal(size=(n,))

In [3]:
start_time = time.time()
re = admm_lasso(X, y, n)
print(time.time() - start_time)

0.043497323989868164


In [4]:
# glmnet
regr = linear_model.ElasticNet(l1_ratio=1, fit_intercept=False, alpha=1)
start_time = time.time()
fit = regr.fit(X, y)
print(time.time() - start_time)

0.007172107696533203


In [5]:
re_glmnet = fit.coef_

beta_mat = [re,  re_glmnet]
print(beta_mat)

[array([-0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.        , -0.        ,
       -0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.        , -0. 

In [6]:
# objective function value
def obj(beta, X, y, tau):
    return 0.5 * np.linalg.norm(y - X.dot(beta)) ** 2 + tau * np.sum(np.abs(beta))

result = [obj(b, X, y, n) for b in beta_mat]
print(result)

[1450.4263570849896, 1450.4254168538605]
